In [2]:
import spacy
nlp = spacy.load("de_core_news_lg")
import json
import os

In [3]:
fixed_path_template = '../07 train models/synth/Llama70B/fixed/split_{}.json'
random_path_template = '../07 train models/synth/Llama70B/random/split_{}.json'

data_dictionary = []

for split_index in range(5):
    path_examples = fixed_path_template.format(split_index)
    with open(path_examples, 'r', encoding="utf-8") as file:
        data_dictionary.extend(json.load(file))

for split_index in range(5):
    path_examples = random_path_template.format(split_index)
    with open(path_examples, 'r', encoding="utf-8") as file:
        data_dictionary.extend(json.load(file))

len(data_dictionary)

17375

In [4]:
def get_tags_for_substring(text, start, end):
    doc = nlp(text)
    substring_tags = [(token.pos_, token.idx, token.text) for token in doc if token.idx >= start and token.idx < end]
    return substring_tags

In [5]:
def has_aspect_term_of_invalid_pos_tags(ex):
    for tag in [t for t in ex["tags"] if t["type"] == "label-explicit"]:
        part_of_speech_tags_of_aspect_term = get_tags_for_substring(ex["text"], tag["start"], tag["end"])
        if len(part_of_speech_tags_of_aspect_term) == 1:
            if part_of_speech_tags_of_aspect_term[0][0] in ["ADJ", "ADV", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "PART", "PRON", "SCONJ", "VERB"]:
                return True
    return False

In [6]:
has_aspect_term_of_invalid_pos_tags([ex for ex in data_dictionary][0])

False

In [7]:
# Diese analysieren ADJ, ADV, AUX, CONJ, CCONJ, DET, INTJ, PART, PRON, SCONJ, VERB

In [9]:
k = 0
for entry in data_dictionary:
    for tag in [t for t in entry["tags"] if t["type"] == "label-explicit"]:
        potags = get_tags_for_substring(
            entry["text"], tag["start"], tag["end"])
        
        if all(tag in ["ADJ", "ADV", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "PART", "PRON", "SCONJ", "VERB"] for tag in list(set(t[0] for t in potags))):
            k += 1
            print(k, tag["text"], potags, entry["llm_prediction_raw"])

        if len(tag["text"]) == 0:
            k += 1
            print(k, tag["text"], potags, entry["llm_prediction_raw"])

1 nicht [('PART', 36, 'nicht')] Sie gaben uns zu verstehen, dass es <aspect-term aspect="FOOD" polarity="NEUTRAL">nicht</aspect-term> wie das Original schmeckt.
2 Wir kamen recht spät [('PRON', 0, 'Wir'), ('VERB', 4, 'kamen'), ('ADV', 10, 'recht'), ('ADV', 16, 'spät')] <aspect-term aspect="SERVICE" polarity="NEUTRAL">Wir kamen recht spät</aspect-term>, aber die Bedienung war sehr nett.
3 Servicepersonal [('ADV', 0, 'Servicepersonal')] <aspect-term aspect="SERVICE" polarity="POSITIVE">Servicepersonal</aspect-term> sehr zuvorkommend und hilfsbereit.
4 freundlich [('ADV', 5, 'freundlich')] Sehr <aspect-term aspect="SERVICE" polarity="POSITIVE">freundlich</aspect-term>, aber etwas langsam.
5 neutrales [('ADJ', 57, 'neutrales')] Kleiner <aspect-term aspect="FOOD" polarity="NEGATIVE">Appetizer</aspect-term>, sehr schnell serviert, aber auch sehr <aspect-term aspect="FOOD" polarity="NEUTRAL">neutrales</aspect-term> Geschmackserlebnis.
6 karg [('ADV', 14, 'karg')] Ambiente eher <aspect-term as